In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('updated2.csv')

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 3420):
  resume = re.sub('[^a-zA-Z]', ' ', dataset['combined'][i])
  resume = resume.lower()
  resume = resume.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  resume = [ps.stem(word) for word in resume if not word in set(all_stopwords)]
  resume = ' '.join(resume)
  corpus.append(resume)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
y

array([ 8,  8, 18, ..., 41, 19, 19])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [9]:
#from sklearn.naive_bayes import GaussianNB
#classifier = GaussianNB()
#classifier.fit(X_train, y_train)

In [10]:
#from sklearn.ensemble import RandomForestClassifier
#classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
#classifier.fit(X_train, y_train)

In [11]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [12]:
le.inverse_transform(y_test)

array(['NYC HOUSING AUTHORITY', 'DEPT OF ENVIRONMENT PROTECTION',
       'DEPT OF ENVIRONMENT PROTECTION', 'DEPARTMENT OF TRANSPORTATION',
       'DEPARTMENT OF TRANSPORTATION', 'DEPT OF HEALTH/MENTAL HYGIENE',
       'DEPARTMENT OF TRANSPORTATION', 'DEPARTMENT OF SANITATION',
       'NYC HOUSING AUTHORITY', 'DEPARTMENT OF BUSINESS SERV.',
       'DEPT OF ENVIRONMENT PROTECTION', 'DEPT OF HEALTH/MENTAL HYGIENE',
       "ADMIN FOR CHILDREN'S SVCS", 'DEPT OF ENVIRONMENT PROTECTION',
       'LAW DEPARTMENT', "ADMIN FOR CHILDREN'S SVCS",
       'DEPT OF HEALTH/MENTAL HYGIENE', 'DEPT OF HEALTH/MENTAL HYGIENE',
       'DEPT OF ENVIRONMENT PROTECTION', 'TAXI & LIMOUSINE COMMISSION',
       "ADMIN FOR CHILDREN'S SVCS", 'DEPT OF HEALTH/MENTAL HYGIENE',
       'DEPT OF ENVIRONMENT PROTECTION', 'DEPT OF HEALTH/MENTAL HYGIENE',
       'DEPT OF HEALTH/MENTAL HYGIENE', 'DEPT OF INFO TECH & TELECOMM',
       'DEPT OF ENVIRONMENT PROTECTION', 'OFFICE OF EMERGENCY MANAGEMENT',
       'HOUSING PRESERVAT

In [13]:
y_pred = classifier.predict(X_test)
print(np.concatenate((le.inverse_transform(y_pred).reshape(len(y_pred),1), le.inverse_transform(y_test).reshape(len(y_test),1)),1))

[['NYC HOUSING AUTHORITY' 'NYC HOUSING AUTHORITY']
 ['DEPT OF ENVIRONMENT PROTECTION' 'DEPT OF ENVIRONMENT PROTECTION']
 ['DEPT OF ENVIRONMENT PROTECTION' 'DEPT OF ENVIRONMENT PROTECTION']
 ...
 ['HOUSING PRESERVATION & DVLPMNT' 'HOUSING PRESERVATION & DVLPMNT']
 ['DEPT OF ENVIRONMENT PROTECTION' 'DEPT OF ENVIRONMENT PROTECTION']
 ['HUMAN RIGHTS COMMISSION' 'HUMAN RIGHTS COMMISSION']]


In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, roc_auc_score
cm = confusion_matrix(le.inverse_transform(y_test), le.inverse_transform(y_pred))
print(cm)
accuracy_score(y_test, y_pred)

[[27  0  0 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 2  0  0 ...  3  0  0]
 [ 0  0  0 ...  0  3  0]
 [ 0  0  0 ...  0  0 11]]


0.8947368421052632

In [15]:
precision_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.9089428249102017

In [16]:
recall_score(y_test, y_pred, average='weighted')

0.8947368421052632

In [17]:
f1_score(y_test, y_pred, average='weighted')

0.8856137615511614

In [18]:
#pred = classifier.predict_proba(X_test)
#log_loss(y_test,y_pred) 

In [19]:
#roc_auc_score(y_test, y_pred, average = 'weighted', multi_class= 'ovr')